# InSAR time series analysis with HyP3 and MintPy

This notebook shows how to prepare hyp3 products for a time-series analysis with MintPy. It requires `hyp3_sdk`.

+ run `conda install --yes -c conda-forge hyp3_sdk ipywidgets` to install `hyp3_sdk`
+ check the [installation page](https://github.com/insarlab/MintPy/blob/main/docs/installation.md) to install `MintPy`
+ check the tutorial (https://github.com/insarlab/MintPy-tutorial/blob/main/smallbaselineApp_hyp3.ipynb) to find out how to do a time series analysis with hyp3 products
+ In this tutorial, we use a 2019 Ridgecrest, CA earthquake (https://earthquake.usgs.gov/storymap/index-ridgecrest.html) data set as the example.

## 0. Initial setup of the notebook

The cell below performs the intial setup of the notebook and must be **run every time the notebook (re)starts**. It imports necessary modules and defines the processing location.

In [ ]:
import re
import zipfile
import numpy as np
import hyp3_sdk
from pathlib import Path
from osgeo import gdal

Define needed paths and create analysis directories

In [ ]:
proj_name = 'test_prep_ts_hyp3'
proj_dir = Path.cwd().joinpath(proj_name)
hyp3_dir = proj_dir.joinpath('hyp3')
dirs = [proj_dir, hyp3_dir]

for dir in dirs:
    if not dir.is_dir():
        dir.mkdir()

## 1. Search SLC filenames

Get the SLC filename pairs to be used for InSAR processing with HyP3. 

One could use the ASF vertex (https://search.asf.alaska.edu) to find these.

We define our example pairs below:

In [ ]:
pairs = [
  ('S1A_IW_SLC__1SDV_20190610T135156_20190610T135223_027618_031DEF_C70F', 'S1A_IW_SLC__1SDV_20190622T135157_20190622T135224_027793_03232E_0388'),
  ('S1A_IW_SLC__1SDV_20190610T135156_20190610T135223_027618_031DEF_C70F', 'S1A_IW_SLC__1SDV_20190704T135158_20190704T135225_027968_032877_1C4D'),
  ('S1A_IW_SLC__1SDV_20190622T135157_20190622T135224_027793_03232E_0388', 'S1A_IW_SLC__1SDV_20190704T135158_20190704T135225_027968_032877_1C4D'),
  ('S1A_IW_SLC__1SDV_20190622T135157_20190622T135224_027793_03232E_0388', 'S1A_IW_SLC__1SDV_20190716T135159_20190716T135226_028143_032DC3_512B'),
  ('S1A_IW_SLC__1SDV_20190704T135158_20190704T135225_027968_032877_1C4D', 'S1A_IW_SLC__1SDV_20190716T135159_20190716T135226_028143_032DC3_512B'),
  ('S1A_IW_SLC__1SDV_20190704T135158_20190704T135225_027968_032877_1C4D', 'S1A_IW_SLC__1SDV_20190728T135159_20190728T135226_028318_03331A_C807'),
  ('S1A_IW_SLC__1SDV_20190716T135159_20190716T135226_028143_032DC3_512B', 'S1A_IW_SLC__1SDV_20190728T135159_20190728T135226_028318_03331A_C807'),
  ('S1A_IW_SLC__1SDV_20190716T135159_20190716T135226_028143_032DC3_512B', 'S1A_IW_SLC__1SDV_20190809T135200_20190809T135227_028493_033887_08A4'),
  ('S1A_IW_SLC__1SDV_20190728T135159_20190728T135226_028318_03331A_C807', 'S1A_IW_SLC__1SDV_20190809T135200_20190809T135227_028493_033887_08A4'),
  ('S1A_IW_SLC__1SDV_20190728T135159_20190728T135226_028318_03331A_C807', 'S1A_IW_SLC__1SDV_20190821T135201_20190821T135228_028668_033EA0_ADF7'),
  ('S1A_IW_SLC__1SDV_20190809T135200_20190809T135227_028493_033887_08A4', 'S1A_IW_SLC__1SDV_20190821T135201_20190821T135228_028668_033EA0_ADF7'),
  ('S1A_IW_SLC__1SDV_20190809T135200_20190809T135227_028493_033887_08A4', 'S1B_IW_SLC__1SDV_20190815T135119_20190815T135146_017597_0211A9_DE22'),
  ('S1A_IW_SLC__1SDV_20190821T135201_20190821T135228_028668_033EA0_ADF7', 'S1B_IW_SLC__1SDV_20190815T135119_20190815T135146_017597_0211A9_DE22'),
  ('S1A_IW_SLC__1SDV_20190821T135201_20190821T135228_028668_033EA0_ADF7', 'S1B_IW_SLC__1SDV_20190827T135119_20190827T135146_017772_021724_4F37'),
  ('S1B_IW_SLC__1SDV_20190815T135119_20190815T135146_017597_0211A9_DE22', 'S1B_IW_SLC__1SDV_20190827T135119_20190827T135146_017772_021724_4F37'),
]

## 2. Use the [hyp3-sdk](https://github.com/ASFHyP3/hyp3-sdk) to submit jobs for InSAR processing and to download the hyp3 products

In [ ]:
# initialize a HyP3 object
try:
    hyp3 = hyp3_sdk.HyP3(hyp3_sdk.TEST_API)
except hyp3_sdk.exceptions.AuthenticationError:
    hyp3 = hyp3_sdk.HyP3(prompt = True)

insar_jobs = hyp3_sdk.Batch()

# submit InSAR jobs
for pair in pairs:
    reference_granule = pair[0]
    secondary_granule = pair[1]
    insar_jobs += hyp3.submit_insar_job(reference_granule, secondary_granule,
                                        name=proj_name, include_look_vectors=True,
                                        include_dem=True)
# watch InSAR jobs
insar_jobs = hyp3.watch(insar_jobs)

# download and uncompress InSAR results
zip_files = insar_jobs.download_files(proj_dir)

for zip_file in zip_files:
    with zipfile.ZipFile(zip_file) as zip_ref:
        zip_ref.extractall(hyp3_dir)
        zip_file.unlink()

# prepared data structure are:
!tree {hyp3_dir}

## 3. Prepare InSAR files for MintPy time series analysis

By clipping the GeoTIFF files into the same grids using `gdal`.

In [ ]:
fnames = list(hyp3_dir.glob('*/*.tif'))

# determine the smallest area covered by all input files
corners = [gdal.Info(str(f), format='json')['cornerCoordinates'] for f in fnames]
ulx = max(corner['upperLeft'][0] for corner in corners)
uly = min(corner['upperLeft'][1] for corner in corners)
lrx = min(corner['lowerRight'][0] for corner in corners)
lry = max(corner['lowerRight'][1] for corner in corners)

# clip the files
regex = '(?<=_)(corr|unw_phase|water_mask|dem|lv_theta)(?=.tif)'
for fname in fnames:
    if re.search(regex, str(fname)):
        fname_out = fname.parent.joinpath(Path(f"{fname.stem}_clip{fname.suffix}"))
        gdal.Translate(destName=str(fname_out), srcDS=str(fname), projWin=[ulx, uly, lrx, lry])
        fname.unlink()
        
# remove the *.xml, png, and *.kmz files
for pattern in ["xml","png","kmz","md.txt"]:
    unneeded_files = hyp3_dir.glob(f"*/*.{pattern}")
    for file in unneeded_files:
        file.unlink()

# show the data structure
!tree {hyp3_dir}